In [1]:
from transformers import RobertaForMaskedLM, RobertaTokenizer
import torch
from tqdm import tqdm

In [2]:
model = RobertaForMaskedLM.from_pretrained('roberta-base')
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

In [3]:
import os
import pandas as pd
from random import choice

import nltk

In [4]:
path_to_intents = os.path.join('data', 'raw')
intents = os.listdir(path_to_intents)
get_path = lambda x: os.path.join('data', 'raw', x, x + ".csv")

In [5]:
os.chdir('helpers')
from analyze import questions, entities, get_data
data = get_data()
os.chdir('..')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dhruv\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
from collections import defaultdict
text = defaultdict(list)
encoded_text = defaultdict(list)

ans = defaultdict(list)
response = defaultdict(list)

scores = defaultdict(list)

for (_questions, intent, entity) in zip(questions, intents, entities):
    question = choice(_questions)
    
    for row in data[intent]['df']['text']:
        text[intent].append(row.strip() + '. ' + question.strip())
        
    for row in data[intent]['df'][entity]:
        ans[intent].append((str(row)).split())

In [8]:
for intent in intents:
    for row in text[intent]:
        encoded_text[intent].append(torch.tensor([tokenizer.encode(row, add_special_tokens=True)]))

In [9]:
encoded_text['BookRestaurant'][0]

tensor([[6298,   20, 2367,  953, 1437, 2391,   11, 2808,   13, 5996,    4, 4820,
          116]])

In [10]:
text['BookRestaurant'][0]

'book The Middle East  restaurant in IN for noon. Where?'

In [11]:
ans['BookRestaurant'][0]

['The', 'Middle', 'East', 'in', 'IN']

In [ ]:
with torch.no_grad():
    for intent in intents:
        for i, row in tqdm(enumerate(encoded_text[intent])):
            out = model(row)
            actual_out = model((torch.argmax(out[0][0], dim=1)).view(1, -1))
            response[intent].append(tokenizer.decode(torch.argmax(actual_out[0][0], dim=1).tolist()).split())
            
            if len(response[intent][i]) > 4:
                score = nltk.translate.bleu_score.sentence_bleu([ans[intent][i]], response[intent][i])
            else:
                weights = [1/len(response[intent][i]) for x in range(len(response[intent][i]))]
                score = nltk.translate.bleu_score.sentence_bleu([ans[intent][i]], response[intent][i], weights = tuple(weights))
            
            scores[intent].append(score)
            

0it [00:00, ?it/s]A sequence with no special tokens has been passed to the RoBERTa model. This model requires special tokens in order to work. Please specify add_special_tokens=True in your tokenize.encode()or tokenizer.convert_tokens_to_ids().
C:\Users\dhruv\Anaconda3\lib\site-packages\nltk\translate\bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\dhruv\Anaconda3\lib\site-packages\nltk\translate\bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
1it [00:00,  3.13it/s]A sequence with no special tokens has been passed to the 

29it [00:08,  3.35it/s]A sequence with no special tokens has been passed to the RoBERTa model. This model requires special tokens in order to work. Please specify add_special_tokens=True in your tokenize.encode()or tokenizer.convert_tokens_to_ids().
30it [00:08,  3.36it/s]A sequence with no special tokens has been passed to the RoBERTa model. This model requires special tokens in order to work. Please specify add_special_tokens=True in your tokenize.encode()or tokenizer.convert_tokens_to_ids().
31it [00:09,  3.51it/s]A sequence with no special tokens has been passed to the RoBERTa model. This model requires special tokens in order to work. Please specify add_special_tokens=True in your tokenize.encode()or tokenizer.convert_tokens_to_ids().
32it [00:09,  3.47it/s]A sequence with no special tokens has been passed to the RoBERTa model. This model requires special tokens in order to work. Please specify add_special_tokens=True in your tokenize.encode()or tokenizer.convert_tokens_to_ids().


62it [00:17,  4.05it/s]A sequence with no special tokens has been passed to the RoBERTa model. This model requires special tokens in order to work. Please specify add_special_tokens=True in your tokenize.encode()or tokenizer.convert_tokens_to_ids().
63it [00:17,  4.10it/s]A sequence with no special tokens has been passed to the RoBERTa model. This model requires special tokens in order to work. Please specify add_special_tokens=True in your tokenize.encode()or tokenizer.convert_tokens_to_ids().
64it [00:17,  3.77it/s]A sequence with no special tokens has been passed to the RoBERTa model. This model requires special tokens in order to work. Please specify add_special_tokens=True in your tokenize.encode()or tokenizer.convert_tokens_to_ids().
65it [00:18,  3.77it/s]A sequence with no special tokens has been passed to the RoBERTa model. This model requires special tokens in order to work. Please specify add_special_tokens=True in your tokenize.encode()or tokenizer.convert_tokens_to_ids().


95it [00:26,  4.00it/s]A sequence with no special tokens has been passed to the RoBERTa model. This model requires special tokens in order to work. Please specify add_special_tokens=True in your tokenize.encode()or tokenizer.convert_tokens_to_ids().
96it [00:26,  3.95it/s]A sequence with no special tokens has been passed to the RoBERTa model. This model requires special tokens in order to work. Please specify add_special_tokens=True in your tokenize.encode()or tokenizer.convert_tokens_to_ids().
97it [00:26,  3.75it/s]A sequence with no special tokens has been passed to the RoBERTa model. This model requires special tokens in order to work. Please specify add_special_tokens=True in your tokenize.encode()or tokenizer.convert_tokens_to_ids().
98it [00:26,  3.84it/s]A sequence with no special tokens has been passed to the RoBERTa model. This model requires special tokens in order to work. Please specify add_special_tokens=True in your tokenize.encode()or tokenizer.convert_tokens_to_ids().


128it [00:34,  3.40it/s]A sequence with no special tokens has been passed to the RoBERTa model. This model requires special tokens in order to work. Please specify add_special_tokens=True in your tokenize.encode()or tokenizer.convert_tokens_to_ids().
129it [00:34,  3.54it/s]A sequence with no special tokens has been passed to the RoBERTa model. This model requires special tokens in order to work. Please specify add_special_tokens=True in your tokenize.encode()or tokenizer.convert_tokens_to_ids().
130it [00:35,  3.80it/s]A sequence with no special tokens has been passed to the RoBERTa model. This model requires special tokens in order to work. Please specify add_special_tokens=True in your tokenize.encode()or tokenizer.convert_tokens_to_ids().
131it [00:35,  3.92it/s]A sequence with no special tokens has been passed to the RoBERTa model. This model requires special tokens in order to work. Please specify add_special_tokens=True in your tokenize.encode()or tokenizer.convert_tokens_to_ids

...

468it [01:49,  4.31it/s]A sequence with no special tokens has been passed to the RoBERTa model. This model requires special tokens in order to work. Please specify add_special_tokens=True in your tokenize.encode()or tokenizer.convert_tokens_to_ids().
469it [01:49,  4.54it/s]A sequence with no special tokens has been passed to the RoBERTa model. This model requires special tokens in order to work. Please specify add_special_tokens=True in your tokenize.encode()or tokenizer.convert_tokens_to_ids().
470it [01:49,  4.71it/s]A sequence with no special tokens has been passed to the RoBERTa model. This model requires special tokens in order to work. Please specify add_special_tokens=True in your tokenize.encode()or tokenizer.convert_tokens_to_ids().
471it [01:49,  4.67it/s]A sequence with no special tokens has been passed to the RoBERTa model. This model requires special tokens in order to work. Please specify add_special_tokens=True in your tokenize.encode()or tokenizer.convert_tokens_to_ids

501it [01:56,  3.92it/s]A sequence with no special tokens has been passed to the RoBERTa model. This model requires special tokens in order to work. Please specify add_special_tokens=True in your tokenize.encode()or tokenizer.convert_tokens_to_ids().
502it [01:56,  4.06it/s]A sequence with no special tokens has been passed to the RoBERTa model. This model requires special tokens in order to work. Please specify add_special_tokens=True in your tokenize.encode()or tokenizer.convert_tokens_to_ids().
503it [01:57,  4.12it/s]A sequence with no special tokens has been passed to the RoBERTa model. This model requires special tokens in order to work. Please specify add_special_tokens=True in your tokenize.encode()or tokenizer.convert_tokens_to_ids().
504it [01:57,  3.73it/s]A sequence with no special tokens has been passed to the RoBERTa model. This model requires special tokens in order to work. Please specify add_special_tokens=True in your tokenize.encode()or tokenizer.convert_tokens_to_ids

534it [02:04,  4.02it/s]A sequence with no special tokens has been passed to the RoBERTa model. This model requires special tokens in order to work. Please specify add_special_tokens=True in your tokenize.encode()or tokenizer.convert_tokens_to_ids().
535it [02:04,  4.38it/s]A sequence with no special tokens has been passed to the RoBERTa model. This model requires special tokens in order to work. Please specify add_special_tokens=True in your tokenize.encode()or tokenizer.convert_tokens_to_ids().
536it [02:05,  4.53it/s]A sequence with no special tokens has been passed to the RoBERTa model. This model requires special tokens in order to work. Please specify add_special_tokens=True in your tokenize.encode()or tokenizer.convert_tokens_to_ids().
537it [02:05,  4.57it/s]A sequence with no special tokens has been passed to the RoBERTa model. This model requires special tokens in order to work. Please specify add_special_tokens=True in your tokenize.encode()or tokenizer.convert_tokens_to_ids

567it [02:11,  4.62it/s]A sequence with no special tokens has been passed to the RoBERTa model. This model requires special tokens in order to work. Please specify add_special_tokens=True in your tokenize.encode()or tokenizer.convert_tokens_to_ids().
568it [02:11,  4.76it/s]A sequence with no special tokens has been passed to the RoBERTa model. This model requires special tokens in order to work. Please specify add_special_tokens=True in your tokenize.encode()or tokenizer.convert_tokens_to_ids().
569it [02:12,  4.66it/s]A sequence with no special tokens has been passed to the RoBERTa model. This model requires special tokens in order to work. Please specify add_special_tokens=True in your tokenize.encode()or tokenizer.convert_tokens_to_ids().
570it [02:12,  4.56it/s]A sequence with no special tokens has been passed to the RoBERTa model. This model requires special tokens in order to work. Please specify add_special_tokens=True in your tokenize.encode()or tokenizer.convert_tokens_to_ids

600it [02:18,  4.33it/s]A sequence with no special tokens has been passed to the RoBERTa model. This model requires special tokens in order to work. Please specify add_special_tokens=True in your tokenize.encode()or tokenizer.convert_tokens_to_ids().
601it [02:19,  4.25it/s]A sequence with no special tokens has been passed to the RoBERTa model. This model requires special tokens in order to work. Please specify add_special_tokens=True in your tokenize.encode()or tokenizer.convert_tokens_to_ids().
602it [02:19,  3.93it/s]A sequence with no special tokens has been passed to the RoBERTa model. This model requires special tokens in order to work. Please specify add_special_tokens=True in your tokenize.encode()or tokenizer.convert_tokens_to_ids().
603it [02:19,  4.06it/s]A sequence with no special tokens has been passed to the RoBERTa model. This model requires special tokens in order to work. Please specify add_special_tokens=True in your tokenize.encode()or tokenizer.convert_tokens_to_ids

In [ ]:
" ".join(response['BookRestaurant'][0])

In [ ]:
for intent in intents:
    print(f"For {intent}, avg. BLEU score is {sum(scores[intent]) / len(scores[intent])}")

In [14]:
_out = sample_sequence(model, 20, encoded_text['BookRestaurant'][0][0], top_p = 0.9)

NameError: name 'sample_sequence' is not defined

In [ ]:
encoded_text['BookRestaurant'][0][0]

In [ ]:
tokenizer

In [ ]:
tokenizer.decode(_out.squeeze(0).tolist())

In [ ]:
tuple([1, 2])

In [15]:
ans['AddToPlaylist']

[['Cita', 'RomÃ¡ntica', '</s>'],
 ['my', 'Pre-Party', 'R&B', 'Jams', '</s>'],
 ['Trapeo', '</s>'],
 ['my', 'night', 'out', '</s>'],
 ['my', 'Digster', 'Future', 'Hits', '</s>'],
 ['my', 'the', 'piano', 'bar', '</s>'],
 ['cleaning', 'the', 'house', '</s>'],
 ['my', 'Indie', 'EspaÃ±ol', '</s>'],
 ['New', 'Indie', 'Mix', '</s>'],
 ['radar', 'latino', '</s>'],
 ['Latin', 'Pop', 'Rising', '</s>'],
 ['Dcode2016', '</s>'],
 ['my', 'fairy', 'tales', '</s>'],
 ['New', 'Indie', 'Mix', '</s>'],
 ['i', 'love', 'my', 'neo', 'soul', '</s>'],
 ['my', 'Digster', 'Future', 'Hits', '</s>'],
 ['my', 'women', 'of', 'country', '</s>'],
 ['baladas', 'romÃ¡nticas', '</s>'],
 ['my', 'bossa', 'nova', 'dinner', '</s>'],
 ['my', 'legendary', 'guitar', 'solos', '</s>'],
 ['my', 'this', 'is', 'prince', '</s>'],
 ["dena's", 'Baroque', '50', 'Spotify', 'Picks', '</s>'],
 ['Sleepytime', '</s>'],
 ['my', 'La', 'Mejor', 'MÃºsica', 'Dance', '2017', '</s>'],
 ['year', 'in', 'metal', '2016', '</s>'],
 ['psychedelic', 'roc